In [1]:
!pip install timm
# 代码中mixup的使用方法值得借鉴

Looking in indexes: https://pypi.tuna.tsinghua.edu.cn/simple


In [9]:
import torch
from torch import nn, optim
from torchvision import transforms
import timm

from sklearn.model_selection import KFold
import numpy as np
import pandas as pd
from PIL import Image
import matplotlib.pyplot as plt
import time
import os
from d2l import torch as d2l

# 数据分割

In [10]:
dataset_path = '/home/data/datasets/competitions/classify-leaves'
train_path = os.path.join(dataset_path, 'train.csv')
test_path = os.path.join(dataset_path, 'test.csv')
train_csv = pd.read_csv(train_path)

leaves_labels = sorted(list(set(train_csv['label'])))
n_classes = len(leaves_labels)
class_to_num = dict(zip(leaves_labels, range(n_classes)))
num_to_class = {v : k for k, v in class_to_num.items()}

# def train_test_split(data, train_size=.75):
#     """ 分割数据集 旧的方法
#     return: train_data_df, valid_data_df
#     """
#     train_len = int(len(data) * train_size)
#     indexs = np.arange(len(data))
#     np.random.shuffle(indexs)

#     return data.loc[indexs[:train_len]].reset_index(drop=True),\
#            data.loc[indexs[train_len:]].reset_index(drop=True)

# train_csv, valid_csv = train_test_split(train_csv)

# 读取数据

In [11]:
class ReadData(torch.utils.data.Dataset):
    def __init__(self, csv_data, transform=None):
        super(ReadData, self).__init__()
        self.dataset_path = '/home/data/datasets/competitions/classify-leaves'
        self.data = csv_data
        self.transform = transform
        
    def __getitem__(self, idx):
        img = Image.open(os.path.join(self.dataset_path, self.data.loc[idx, "image"]))
        label = class_to_num[self.data.loc[idx, "label"]]
        
        if self.transform:
            img = self.transform(img)
        
        return img, label
        
    def __len__(self):
        return len(self.data)
    
train_transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.RandomHorizontalFlip(p=.5),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

valid_transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

def kfold(data, k=5):
    """ K折交叉验证 """
    
    KF = KFold(n_splits=k, shuffle=False)
    for train_idxs, test_idxs in KF.split(data):
        train_data = data.loc[train_idxs].reset_index(drop=True)
        valid_data = data.loc[test_idxs].reset_index(drop=True)
        train_iter = torch.utils.data.DataLoader(
            ReadData(train_data, train_transform), batch_size=64,
            shuffle=True, num_workers=d2l.get_dataloader_workers(), pin_memory=True
        )

        valid_iter = torch.utils.data.DataLoader(
            ReadData(valid_data, valid_transform), batch_size=64,
            shuffle=True, num_workers=d2l.get_dataloader_workers(), pin_memory=True
        )
        
        yield train_iter, valid_iter

# 数据增强

In [12]:
def mixup_data(x, y, alpha=1.0, use_cuda=True):
    """ Mixup 数据增强 -> 随机叠加两张图像 """
    if alpha > 0:
        lam = np.random.beta(alpha, alpha)
    else:
        lam = 1

    batch_size = x.size()[0]
    # 这里和下面一样，都是使用一个batch的数据随机替换到另一个batch中
    if use_cuda:
        index = torch.randperm(batch_size).cuda()
    else:
        index = torch.randperm(batch_size)

    mixed_x = lam * x + (1 - lam) * x[index, :]
    y_a, y_b = y, y[index]
    return mixed_x, y_a, y_b, lam

In [13]:
def rand_bbox(size, lam):
    """ 随机裁剪 """
    W, H = size[2], size[3]
    cut_rat = np.sqrt(1. - lam)
    cut_w = np.int(W * cut_rat)
    cut_h = np.int(H * cut_rat)

    # uniform
    cx = np.random.randint(W)
    cy = np.random.randint(H)

    bbx1 = np.clip(cx - cut_w // 2, 0, W)
    bby1 = np.clip(cy - cut_h // 2, 0, H)
    bbx2 = np.clip(cx + cut_w // 2, 0, W)
    bby2 = np.clip(cy + cut_h // 2, 0, H)

    return bbx1, bby1, bbx2, bby2

def cutmix_data(x, y, alpha=1.0, use_cuda=True):
    """ Cutmix 数据增强 -> 随机对主图像进行裁剪, 加上噪点图像
    W: 添加裁剪图像宽
    H: 添加裁剪图像高
    """
    if alpha > 0:
        lam = np.random.beta(alpha, alpha)
    else:
        lam = 1

    batch_size = x.size()[0]
    if use_cuda:
        index = torch.randperm(batch_size).cuda()
    else:
        index = torch.randperm(batch_size)
    
    bbx1, bby1, bbx2, bby2 = rand_bbox(x.size(), lam)
    x[:, :, bbx1:bbx2, bby1:bby2] = x[index, :, bbx1:bbx2, bby1:bby2]
    lam = 1 - ((bbx2 - bbx1) * (bby2 - bby1) / (x.size()[-1] * x.size()[-2]))
    y_a, y_b = y, y[index]
    
    return x, y_a, y_b, lam

In [29]:
def get_models(k=5):
    models = {}
    for mk in range(k):
        model = timm.create_model("resnest50d_4s2x40d", True, drop_rate=.5)
        # 在特征提取层之后添加一个mlp用于微调
        model.fc = nn.Sequential(
            nn.Linear(model.fc.in_features, 512),
            nn.ReLU(inplace=True),
            nn.Dropout(.3),
            nn.Linear(512, len(num_to_class))
        )
        # for param in model.layer4.parameters():
        #     if isinstance(param, nn.Conv2d):
        #         nn.init.xavier_normal_(param.weight)
        # for param in model.fc.parameters():
        #     if isinstance(param, nn.Linear):
        #         nn.init.kaiming_normal_(param.weight)
        # model.load_state_dict(torch.load(f"../input/resnest50/Resnest50d.pth"))
        # 固定前面几层
        # 这里是只固定了前六层
        for i, param in enumerate(model.children()):
            if i == 6:
                break
            param.requires_grad = False

        model.cuda()

        opt = optim.AdamW(filter(lambda p: p.requires_grad, model.parameters()), lr=1e-5)
        scheduler = optim.lr_scheduler.CosineAnnealingWarmRestarts(opt, 10, T_mult=2)
        models[f"model_{mk}"] = {
            "model": model,
            "opt": opt,
            "scheduler": scheduler,
            "last_acc": .97
        }
    
    return models

# 一共集成了5个模型，和交叉验证的份数一致
models = get_models()

In [17]:
def mixup_criterion(pred, y_a, y_b, lam):
    c = nn.CrossEntropyLoss()
    return lam * c(pred, y_a) + (1 - lam) * c(pred, y_b)
criterion = mixup_criterion

# 训练

In [30]:
def train_model():
    for epoch in range(10):
        flod_train_acc = []
        flod_valid_acc = []
        for k, (train_iter, valid_iter) in enumerate(kfold(train_csv, 5)):
            model = models[f"model_{k}"]["model"]
            opt = models[f"model_{k}"]["opt"]
            scheduler = models[f"model_{k}"]["scheduler"]
            s = time.time()
            model.train()
            train_loss = []
            train_acc = 0
            length = 0
            for x, y in train_iter:
                x, y = x.cuda(), y.cuda()
                random_num = np.random.random()
                # 随机选取mixup的方式
                if random_num <= 1/3:
                    x, y_a, y_b, lam = mixup_data(x, y, use_cuda=True)
                elif random_num <= 2/3:
                    x, y_a, y_b, lam = cutmix_data(x, y, use_cuda=True)
                else:
                    x, y_a, y_b, lam = mixup_data(x, y, alpha=0, use_cuda=True)
                x, y_a, y_b = map(torch.autograd.Variable, (x, y_a, y_b))
                output = model(x)
                loss = criterion(output, y_a, y_b, lam)
                train_loss.append(loss.item())
                predict = output.argmax(dim=1)
                length += x.shape[0]
                # 因为是对不同batch之间的进行替换，可以使用下面的方法去求解
                train_acc += lam * (predict == y_a).cpu().sum().item() + \
                            (1 - lam) * (predict == y_b).cpu().sum().item()
                opt.zero_grad()
                loss.backward()
                opt.step()
                scheduler.step()

            model.eval()
            valid_acc = []
            with torch.no_grad():
                for x, y in valid_iter:
                    x, y = x.cuda(), y.cuda()
                    pre_x = model(x)
                    valid_acc.append((pre_x.argmax(1) == y).float().mean().item())

            k_train_ = train_acc / length
            k_valid_ = sum(valid_acc) / len(valid_acc)
            if k_valid_ > models[f"model_{k}"]["last_acc"]:
                torch.save(model.state_dict(), f"./kaggle_leaves_competition_model/Resnest50d_{k}.pth")
                models[f"model_{k}"]["last_acc"] = k_valid_

            response = f"Epoch {epoch + 1}-Fold{k + 1} —— " + \
                    f"Train Loss: {sum(train_loss) / len(train_loss) :.3f}, " + \
                    f"Train Accuracy: {k_train_ * 100 :.2f}%, " + \
                    f"Valid Accuracy: {k_valid_ * 100 :.2f}%, " + \
                    f"Learning Rate: {opt.param_groups[0]['lr'] :.6f}, " + \
                    f"Time Out: {time.time() - s :.1f}s"
            print(response)
            flod_train_acc.append(k_train_)
            flod_valid_acc.append(k_valid_)

        t_accuracy = np.mean(flod_train_acc)
        v_accuracy = np.mean(flod_valid_acc)
        print(f"Epoch {epoch + 1} —— " + \
            f"Train Accuracy: {t_accuracy * 100 :.2f}%, " + \
            f"Valid Accuracy: {v_accuracy * 100 :.2f}%\n")

train_model()

/tmp/ipykernel_708527/4276473864.py:5: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  cut_w = np.int(W * cut_rat)
/tmp/ipykernel_708527/4276473864.py:6: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for 

Epoch 1-Fold1 —— Train Loss: 5.168, Train Accuracy: 0.83%, Valid Accuracy: 3.52%, Learning Rate: 0.000005, Time Out: 81.8s
Epoch 1-Fold2 —— Train Loss: 5.162, Train Accuracy: 0.83%, Valid Accuracy: 2.58%, Learning Rate: 0.000005, Time Out: 80.3s
Epoch 1-Fold3 —— Train Loss: 5.159, Train Accuracy: 0.97%, Valid Accuracy: 4.07%, Learning Rate: 0.000005, Time Out: 80.5s
Epoch 1-Fold4 —— Train Loss: 5.164, Train Accuracy: 0.90%, Valid Accuracy: 3.85%, Learning Rate: 0.000005, Time Out: 80.5s
Epoch 1-Fold5 —— Train Loss: 5.162, Train Accuracy: 0.79%, Valid Accuracy: 3.26%, Learning Rate: 0.000005, Time Out: 80.4s
Epoch 1 —— Train Accuracy: 0.86%, Valid Accuracy: 3.46%

Epoch 2-Fold1 —— Train Loss: 5.128, Train Accuracy: 1.78%, Valid Accuracy: 11.86%, Learning Rate: 0.000005, Time Out: 80.7s
Epoch 2-Fold2 —— Train Loss: 5.112, Train Accuracy: 2.21%, Valid Accuracy: 5.17%, Learning Rate: 0.000005, Time Out: 80.6s
Epoch 2-Fold3 —— Train Loss: 5.102, Train Accuracy: 2.31%, Valid Accuracy: 6.28%,

# TestModel

In [10]:
test_csv = pd.read_csv("../input/classify-leaves/test.csv")

In [11]:
class ReadData(torch.utils.data.Dataset):
    def __init__(self, csv_data, transform=None):
        super(ReadData, self).__init__()
        self.data = csv_data
        self.transform = transform
        
    def __getitem__(self, idx):
        img = Image.open("../input/classify-leaves/" + self.data.loc[idx, "image"])
        
        if self.transform:
            img = self.transform(img)
        
        return img
        
    def __len__(self):
        return len(self.data)

test_iter = torch.utils.data.DataLoader(
    ReadData(test_csv, valid_transform), batch_size=64,
    num_workers=2, pin_memory=True
)

In [12]:
predict = None
with torch.no_grad():
    for x in test_iter:
        x = x.cuda()
        p = torch.zeros((x.size()[0], len(class_to_num)))
        for k in range(5):
            model = models[f"model_{k}"]["model"]
            # 这里也相当于集成学习，集成不同的fold训练出的model
            p += model(x).detach().cpu()
        if predict is None:
            predict = p.argmax(1)
        else:
            predict = torch.cat([predict, p.argmax(1)])

In [13]:
df = pd.read_csv("../input/classify-leaves/sample_submission.csv")
df.label = predict.cpu().numpy()
df.label = df.label.apply(lambda x: num_to_class[x])
df.to_csv("/kaggle/working/result.csv", index=False)